In [1]:
%%capture

# Install the SDK (Uncomment the code if the SDK is not installed before)
!python3 -m pip install 'kfp' --quiet
!python3 -m pip install pandas --upgrade -q

In [ ]:
!pip3 install kfp --upgrade --user

In [3]:
PROJECT_ID = ''
IMAGE_NAME = ''

In [2]:
!docker build . -t gcr.io/$PROJECT_ID/$IMAGE_NAME:latest

invalid argument "gcr.io/camus-mlops/gcr.io/camus-mlops/census_keras@sha256:1585fe7686b9a5a3aa2729d1d4c8a969b1bb36ba099c152e227846a86a3e25fe:latest" for "-t, --tag" flag: invalid reference format
See 'docker build --help'.


In [ ]:
!docker push gcr.io/$PROJECT_ID/$IMAGE_NAME:latest

In [4]:
FULL_IMAGE_NAME = ''

In [14]:
%env FULL_IMAGE_NAME={FULL_IMAGE_NAME}
%env JOB_DIR=''
%env TRAINER_PACKAGE_PATH='trainer'
%env MAIN_TRAINER_MODULE=trainer.task
%env REGION=europe-west1

env: FULL_IMAGE_NAME=gcr.io/camus-mlops/census_keras@sha256:1585fe7686b9a5a3aa2729d1d4c8a969b1bb36ba099c152e227846a86a3e25fe
env: JOB_DIR=''
env: TRAINER_PACKAGE_PATH='trainer'
env: MAIN_TRAINER_MODULE=trainer.task
env: REGION=europe-west1


In [12]:
%%writefile ./pipeline/census_training_pipeline.py

"""Simple KFP with one component."""

import os

import kfp
from kfp import dsl
from kfp.components import func_to_container_op
from kfp.dsl.types import Dict
from kfp.dsl.types import GCPProjectID
from kfp.dsl.types import GCPRegion
from kfp.dsl.types import GCSPath
from kfp.dsl.types import String
from kfp.gcp import use_gcp_secret

FULL_IMAGE_NAME = os.getenv('FULL_IMAGE_NAME')
TRAINER_PACKAGE_PATH = os.getenv('TRAINER_PACKAGE_PATH')
REGION = os.getenv('REGION')
MAIN_TRAINER_MODULE = os.getenv('MAIN_TRAINER_MODULE')
JOB_DIR = os.getenv('JOB_DIR')

# Create Training Component

def census_train_op():
    return dsl.ContainerOp(
        name='Census Training Component',
        image=FULL_IMAGE_NAME,
        arguments=[
        TRAINER_PACKAGE_PATH,
        MAIN_TRAINER_MODULE,
        REGION,
        3.7,
        2.1,
        JOB_DIR        
    ])

@kfp.dsl.pipeline(
    name='Census Keras Training',
    description='The pipeline training a census model.'
)
def census_train(project_id, region):
    """Orchestrates training and deployment of a keras model."""
    census_train_op()

Overwriting ./pipeline/census_training_pipeline.py


In [13]:
!dsl-compile --py pipeline/census_training_pipeline.py --output census_pipeline.yaml